In [ ]:
import pandas as pd 
import numpy as np
from google.colab import drive
# drive.mount('/contact/drive')
# df = pd.read_pickle("/contact/drive/MyDrive/ass2.pickle")
with open('ass2.pickle', 'rb') as file:
    data = pd.read_pickle(file)

first let analaiys this data:

In [ ]:
train = data['train']
dev = data['dev']
test = data['test']
print("The number of featuures the this data set is:", train.shape[1]-1)
print("The number of traing sample of  is:", len(train))
print("The number of Dev samples is: ", len(dev))
print("The number of test samples is:", len(test))
print("traing data")
print(train)
print("dev data")
print(dev)
print("test data")
print(test)
nptrain = np.array(train)
npdev = np.array(dev)
nptest = np.array(test)
totalMin = min(np.min(nptrain[:,:-1]), np.min(npdev[:,:-1]), np.min(nptest[:,:-1]))
totalMax = max(np.max(nptrain[:,:-1]), np.max(npdev[:,:-1]), np.max(nptest[:,:-1]))
print("The range of all the features is:", totalMin, "-", totalMax)
check = np.min(train.target)
mintarget = min( np.min(train.target),  np.min(dev.target),  np.min(test.target))
maxtarget = max( np.min(train.target),  np.min(dev.target),  np.min(test.target))
print("The range of all the targets is:", totalMin, "-", totalMax)


The number of featuures the this data set is: 42
The number of traing sample of  is: 40533
The number of Dev samples is:  13512
The number of test samples is: 13512
traing data
       f0  f1  f2  f3  f4  f5  f6  f7  f8  f9  ...  f33  f34  f35  f36  f37  \
51905   1   0   0   0   0   0   2   1   2   2  ...    0    0    0    2    0   
52612   0   0   0   0   0   0   2   1   0   0  ...    0    0    0    2    0   
61699   2   1   2   1   1   0   2   2   0   0  ...    0    0    0    1    0   
6291    0   0   0   0   0   0   0   0   0   0  ...    0    0    0    2    0   
17484   0   0   0   0   0   0   1   1   2   0  ...    0    0    0    2    1   
...    ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ...  ...  ...  ...  ...  ...   
9379    1   1   0   0   0   0   1   0   0   0  ...    0    0    0    2    0   
40261   1   2   0   0   0   0   0   0   0   0  ...    0    0    0    1    2   
23630   0   0   0   0   0   0   1   0   0   0  ...    0    0    0    2    2   
7937    1   0   0   0   0   0   2

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import f1_score, make_scorer
from xgboost import XGBClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import PassiveAggressiveClassifier


In [3]:
# Load the data
data_dict = pd.read_pickle('ass2.pickle')
train_data = data_dict['train']
data = pd.DataFrame(train_data)
devData = pd.DataFrame(data_dict['dev'])
testData = pd.DataFrame(data_dict['test'])

In [4]:
# Split the data into X (features) and y (target)
train = pd.concat([data, devData])
X_train = train.drop(columns='target')
y_train = train['target']
X_test = testData.drop(columns='target')
y_test = testData['target']
# Split the data into train and test sets
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

First we going to check the model below with minimal parametr so we can choose the models that we want to conti

In [5]:
models = [
    {
        'name' : 'BernoulliNB',
        'model': BernoulliNB(),
        'params':{'force_alpha':[True]}
    },
    {
        'name': 'PassiveAggressiveClassifier',
        'model':PassiveAggressiveClassifier(),
        'params':{'max_iter':[1000], 'random_state':[0]}
    },
    {
        'name': 'DecisionTree',
        'model': DecisionTreeClassifier(),
        'params': {'max_depth': [3], 'min_samples_split': [2]}
    },
    {
        'name': 'RandomForest',
        'model': RandomForestClassifier(),
        'params': {'n_estimators': [10], 'max_depth': [3], 'min_samples_split': [2]}
    },
    {
        'name': 'LogisticRegression',
        'model': LogisticRegression(),
        'params': {'C': [ 0.1], 'solver': ['liblinear']}
    },
    {
        'name': 'KNeighbors',
        'model': KNeighborsClassifier(),
        'params': {'n_neighbors': [3], 'weights': ['distance']}
    },
    {
        'name': 'SVM',
        'model': SVC(),
        'params': {'C': [10]}
    },
    {
        'name': 'XGBoost',
        'model': XGBClassifier(),
        'params': {
            'n_estimators': [200],
            'learning_rate': [0.2],
            'max_depth': [7],
            'subsample': [1],
            'colsample_bytree': [1],
        }
    }    
]

# Function to find the best model and hyperparameters
def find_best_model(X_train, y_train, models):
    best_score = 0
    best_model = None
    best_params = None
    
    # Use StratifiedKFold for cross-validation
    cv = StratifiedKFold(n_splits=5)
    
    # Use F1 score as the evaluation metric
    f1_scorer = make_scorer(f1_score, average='weighted')
    
    for m in models:
        grid_search = GridSearchCV(m['model'], m['params'], cv=cv, scoring=f1_scorer, return_train_score=False, verbose=3)
        grid_search.fit(X_train, y_train)
        
        if grid_search.best_score_ > best_score:
            best_score = grid_search.best_score_
            best_model = m['name']
            best_params = grid_search.best_params_
    
    return best_model, best_params, best_score

# Find the best model and hyperparameters
best_model, best_params, best_score = find_best_model(X_train, y_train, models)

print("Best model: ", best_model)
print("Best parameters: ", best_params)
print("Best F1 score: ", best_score)


Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 1/5] END ..................force_alpha=True;, score=0.541 total time=   0.1s
[CV 2/5] END ..................force_alpha=True;, score=0.542 total time=   0.1s
[CV 3/5] END ..................force_alpha=True;, score=0.545 total time=   0.1s
[CV 4/5] END ..................force_alpha=True;, score=0.543 total time=   0.2s
[CV 5/5] END ..................force_alpha=True;, score=0.542 total time=   0.2s
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 1/5] END .....max_iter=1000, random_state=0;, score=0.512 total time=   0.8s
[CV 2/5] END .....max_iter=1000, random_state=0;, score=0.550 total time=   0.6s
[CV 3/5] END .....max_iter=1000, random_state=0;, score=0.518 total time=   0.8s
[CV 4/5] END .....max_iter=1000, random_state=0;, score=0.447 total time=   0.5s
[CV 5/5] END .....max_iter=1000, random_state=0;, score=0.528 total time=   0.4s
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 1/5] END 

We saw that the best model is XGBoost,

but we going to still going to study the models: DecisionTree, RandomForest, LogisticRegression, KNeighbors, SVM and XGBoost with other parameters

maybe we can make better result from those models.

In [ ]:
# Define a list of models and their hyperparameters
models = [
    {
        'name': 'DecisionTree',
        'model': DecisionTreeClassifier(),
        'params': {'max_depth': [3, 5, 7, 9, 11], 'min_samples_split': [2, 5, 10, 20]}
    },
    {
        'name': 'RandomForest',
        'model': RandomForestClassifier(),
        'params': {'n_estimators': [10, 50, 100], 'max_depth': [3, 5, 7, 9, 11], 'min_samples_split': [2, 5, 10, 20]}
    },
    {
        'name': 'LogisticRegression',
        'model': LogisticRegression(),
        'params': {'C': [0.001, 0.01, 0.1, 1, 10, 100], 'solver': ['newton-cg', 'liblinear']}
    },
    {
        'name': 'KNeighbors',
        'model': KNeighborsClassifier(),
        'params': {'n_neighbors': [3, 5, 7, 9, 11], 'weights': ['uniform', 'distance']}
    }
    
]

# Function to find the best model and hyperparameters
def find_best_model(X_train, y_train, models):
    best_score = 0
    best_model = None
    best_params = None
    
    # Use StratifiedKFold for cross-validation
    cv = StratifiedKFold(n_splits=5)
    
    # Use F1 score as the evaluation metric
    f1_scorer = make_scorer(f1_score, average='weighted')
    
    for m in models:
        grid_search = GridSearchCV(m['model'], m['params'], cv=cv, scoring=f1_scorer, return_train_score=False, verbose=3)
        grid_search.fit(X_train, y_train)
        
        if grid_search.best_score_ > best_score:
            best_score = grid_search.best_score_
            best_model = m['name']
            best_params = grid_search.best_params_
    
    return best_model, best_params, best_score

# Find the best model and hyperparameters
best_model, best_params, best_score = find_best_model(X_train, y_train, models)

print("Best model: ", best_model)
print("Best parameters: ", best_params)
print("Best F1 score: ", best_score)


Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV 1/5] END ..max_depth=3, min_samples_split=2;, score=0.576 total time=   0.1s
[CV 2/5] END ..max_depth=3, min_samples_split=2;, score=0.576 total time=   0.1s
[CV 3/5] END ..max_depth=3, min_samples_split=2;, score=0.575 total time=   0.1s
[CV 4/5] END ..max_depth=3, min_samples_split=2;, score=0.574 total time=   0.1s
[CV 5/5] END ..max_depth=3, min_samples_split=2;, score=0.574 total time=   0.1s
[CV 1/5] END ..max_depth=3, min_samples_split=5;, score=0.576 total time=   0.1s
[CV 2/5] END ..max_depth=3, min_samples_split=5;, score=0.576 total time=   0.1s
[CV 3/5] END ..max_depth=3, min_samples_split=5;, score=0.575 total time=   0.1s
[CV 4/5] END ..max_depth=3, min_samples_split=5;, score=0.574 total time=   0.1s
[CV 5/5] END ..max_depth=3, min_samples_split=5;, score=0.574 total time=   0.1s
[CV 1/5] END .max_depth=3, min_samples_split=10;, score=0.576 total time=   0.1s
[CV 2/5] END .max_depth=3, min_samples_split=10

In [ ]:
# Define a list of models and their hyperparameters
models = [
    {
        'name': 'SVM',
        'model': SVC(),
        'params': {
        'C': [10],
        'kernel': ['poly'],
        'coef0': [1, 2],  
        # 'shrinking': [True, False]
        }
    }
]

# Function to find the best model and hyperparameters
def find_best_model(X_train, y_train, models):
    best_score = 0
    best_model = None
    best_params = None
    
    # Use StratifiedKFold for cross-validation
    cv = StratifiedKFold(n_splits=5)
    
    # Use F1 score as the evaluation metric
    f1_scorer = make_scorer(f1_score, average='weighted')
    
    for m in models:
        grid_search = GridSearchCV(m['model'], m['params'], cv=cv, scoring=f1_scorer, return_train_score=False, verbose=3)
        grid_search.fit(X_train, y_train)
        
        if grid_search.best_score_ > best_score:
            best_score = grid_search.best_score_
            best_model = m['name']
            best_params = grid_search.best_params_
    
    return best_model, best_params, best_score

# Find the best model and hyperparameters
best_model, best_params, best_score = find_best_model(X_train, y_train, models)

print("Best model: ", best_model)
print("Best parameters: ", best_params)
print("Best F1 score: ", best_score)


Fitting 5 folds for each of 2 candidates, totalling 10 fits
[CV 1/5] END ........C=10, coef0=1, kernel=poly;, score=0.788 total time= 5.6min
[CV 2/5] END ........C=10, coef0=1, kernel=poly;, score=0.789 total time= 5.7min
[CV 3/5] END ........C=10, coef0=1, kernel=poly;, score=0.787 total time= 5.8min
[CV 4/5] END ........C=10, coef0=1, kernel=poly;, score=0.792 total time= 5.7min
[CV 5/5] END ........C=10, coef0=1, kernel=poly;, score=0.791 total time= 5.7min
[CV 1/5] END ........C=10, coef0=2, kernel=poly;, score=0.790 total time=10.7min
[CV 2/5] END ........C=10, coef0=2, kernel=poly;, score=0.789 total time=10.4min
[CV 3/5] END ........C=10, coef0=2, kernel=poly;, score=0.788 total time=10.5min
[CV 4/5] END ........C=10, coef0=2, kernel=poly;, score=0.793 total time=10.2min
[CV 5/5] END ........C=10, coef0=2, kernel=poly;, score=0.793 total time=10.6min
Best model:  SVM
Best parameters:  {'C': 10, 'coef0': 2, 'kernel': 'poly'}
Best F1 score:  0.7903675253934765


In [ ]:
# Define a list of models and their hyperparameters
models = [
    
    {
        'name': 'XGBoost',
        'model': XGBClassifier(),
        'params': {
            'n_estimators': [50, 100, 200],
            'learning_rate': [0.1, 0.2],
            'max_depth': [5, 7],
            'subsample': [0.7, 1],
            'colsample_bytree': [0.5, 0.7, 1],
        }
    }
]

# Function to find the best model and hyperparameters
def find_best_model(X_train, y_train, models):
    best_score = 0
    best_model = None
    best_params = None
    
    # Use StratifiedKFold for cross-validation
    cv = StratifiedKFold(n_splits=5)
    
    # Use F1 score as the evaluation metric
    f1_scorer = make_scorer(f1_score, average='weighted')
    
    for m in models:
        grid_search = GridSearchCV(m['model'], m['params'], cv=cv, scoring=f1_scorer, return_train_score=False, verbose=3)
        grid_search.fit(X_train, y_train)
        
        if grid_search.best_score_ > best_score:
            best_score = grid_search.best_score_
            best_model = m['name']
            best_params = grid_search.best_params_
    
    return best_model, best_params, best_score

# Find the best model and hyperparameters
best_model, best_params, best_score = find_best_model(X_train, y_train, models)

print("Best model: ", best_model)
print("Best parameters: ", best_params)
print("Best F1 score: ", best_score)


Fitting 5 folds for each of 72 candidates, totalling 360 fits
[CV 1/5] END colsample_bytree=0.5, learning_rate=0.1, max_depth=5, n_estimators=50, subsample=0.7;, score=0.695 total time=  11.5s
[CV 2/5] END colsample_bytree=0.5, learning_rate=0.1, max_depth=5, n_estimators=50, subsample=0.7;, score=0.691 total time=   8.9s
[CV 3/5] END colsample_bytree=0.5, learning_rate=0.1, max_depth=5, n_estimators=50, subsample=0.7;, score=0.687 total time=   8.8s
[CV 4/5] END colsample_bytree=0.5, learning_rate=0.1, max_depth=5, n_estimators=50, subsample=0.7;, score=0.695 total time=   7.2s
[CV 5/5] END colsample_bytree=0.5, learning_rate=0.1, max_depth=5, n_estimators=50, subsample=0.7;, score=0.694 total time=   8.9s
[CV 1/5] END colsample_bytree=0.5, learning_rate=0.1, max_depth=5, n_estimators=50, subsample=1;, score=0.695 total time=   8.0s
[CV 2/5] END colsample_bytree=0.5, learning_rate=0.1, max_depth=5, n_estimators=50, subsample=1;, score=0.692 total time=   6.3s
[CV 3/5] END colsample_by

from the models that we search we found that the model that is the most suitable is XGBoost, 

with the hyperprameters: 'colsample_bytree': 1, 'learning_rate': 0.2, 'max_depth': 7, 'n_estimators': 200, 'subsample': 0.7

now we look if we can search around those parametr and find better result

In [ ]:
models = [
    
    {
        'name': 'XGBoost',
        'model': XGBClassifier(),
        'params': {
            'n_estimators': [200],
            'learning_rate': [0.3, 0.4 ],
            'max_depth': [8, 9],
            'subsample': [0.75, 0.8],
            'colsample_bytree': [0.95, 1],
        }
    }
]

# Function to find the best model and hyperparameters
def find_best_model(X_train, y_train, models):
    best_score = 0
    best_model = None
    best_params = None
    
    # Use StratifiedKFold for cross-validation
    cv = StratifiedKFold(n_splits=5)
    
    # Use F1 score as the evaluation metric
    f1_scorer = make_scorer(f1_score, average='weighted')
    
    for m in models:
        grid_search = GridSearchCV(m['model'], m['params'], cv=cv, scoring=f1_scorer, return_train_score=False, verbose=3)
        grid_search.fit(X_train, y_train)
        
        if grid_search.best_score_ > best_score:
            best_score = grid_search.best_score_
            best_model = m['name']
            best_params = grid_search.best_params_
    
    return best_model, best_params, best_score

# Find the best model and hyperparameters
best_model, best_params, best_score = find_best_model(X_train, y_train, models)

print("Best model: ", best_model)
print("Best parameters: ", best_params)
print("Best F1 score: ", best_score)

Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV 1/5] END colsample_bytree=0.95, learning_rate=0.3, max_depth=8, n_estimators=200, subsample=0.75;, score=0.847 total time= 1.4min
[CV 2/5] END colsample_bytree=0.95, learning_rate=0.3, max_depth=8, n_estimators=200, subsample=0.75;, score=0.844 total time= 1.4min
[CV 3/5] END colsample_bytree=0.95, learning_rate=0.3, max_depth=8, n_estimators=200, subsample=0.75;, score=0.849 total time= 1.5min
[CV 4/5] END colsample_bytree=0.95, learning_rate=0.3, max_depth=8, n_estimators=200, subsample=0.75;, score=0.845 total time= 1.4min
[CV 5/5] END colsample_bytree=0.95, learning_rate=0.3, max_depth=8, n_estimators=200, subsample=0.75;, score=0.847 total time= 1.5min
[CV 1/5] END colsample_bytree=0.95, learning_rate=0.3, max_depth=8, n_estimators=200, subsample=0.8;, score=0.848 total time= 1.4min
[CV 2/5] END colsample_bytree=0.95, learning_rate=0.3, max_depth=8, n_estimators=200, subsample=0.8;, score=0.840 total time= 1.4min
[CV

We use wieghts to balance the data set as preprocessing, it means that the attribute of each data sample is not equal is depened on the frequency of this label in the dataset.

The metric that we check for choosing the best ML model is F1 as asked at the job describtion.

after checking 8 models and we found out that the model with the best result was XGBoost with the parameters:

colsample_bytree: 1, learning_rate: 0.3, max_depth: 9, n_estimators: 200, subsample: 0.8

The afect of each parameters is:
*   colsample_bytree: is the subsample ratio of columns when constructing each tree. Subsampling occurs once for every tree constructed.
*   learning_rate: this parmeter is to preventing over fitting soo if we make the learning rate to big we risking to over fit the model to the data
*   max_depth: Maximum depth of a tree. Increasing this value will make the model more complex and more likely to overfit. and for bigger depth of traing tree is will going to consumes more memory
*   subsample: is the precentge of the data that will be sample prior to growing tree, if the value is to big we risking with overfitting

We choose not to run PCA to reduce the dimention of the data because the computer succeed to do the calculation and we have inoght time to run the models








